### User behaviour

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn-muted")

from sqlalchemy import create_engine, inspect   #presto

In [2]:
sns.set_style("white")

### Conexión a Presto

In [150]:
#setear usuario
usuario={"usuario":"------","contraseña":"-----"}

In [151]:
#crear conexión
url="presto://"+usuario['usuario']+"@hue.peya.co:8889/hive/datalake"
engine = create_engine(url)
connection = engine.connect()

In [7]:
#Definir consulta
query='''
select *
from default.main_cuisine_abr_may
where cdcountry='Argentina'
'''

In [152]:
#Ejecutar la consulta
main_cuisine = pd.read_sql(query,con=connection)
main_cuisine.head()

,userid,cdcountry,month,qty_shops,total_orders,not_weekend,weekend,lunes,martes,miercoles,...,helados,sushi,pollo,milanesas,sandwiches,other,order_discount,coupon,rating,amount
0,12428378,Argentina,4,1,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,4.69,450.00
1,18379938,Argentina,5,2,3,1,2,0,0,0,...,0,0,0,2,0,1,0,0,4.36,1166.67
2,12886415,Argentina,5,1,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,4.84,500.00
3,19287823,Argentina,5,2,2,0,2,0,0,0,...,0,0,0,0,0,0,1,1,4.64,1102.00
4,16545147,Argentina,5,5,6,5,1,3,2,0,...,2,0,0,0,0,2,1,1,4.48,273.17


### Conexión a BQ

In [ ]:
seg_may = pd.io.gbq.read_gbq('''
  select customer_id as userid
        , activity_group as activity_group_may
        , vendor_selection_group as vendor_selection_group_may
        , customer_experience_group as customer_experience_group_may
FROM `bpy---pedidosya.user_leticia_colombo.user_profile_Arg_may` 
where cast(customer_id as numeric)<>0 or customer_id is not null
''', project_id="bpy---pedidosya", dialect='standard')

In [ ]:
seg_may.userid=seg_may.userid.astype('int64') #paso a int el user id porque en la otra tabla está como int

In [ ]:
# Concat de main_cuisine con tabla de BQ segmentos
mcm=main_cuisine[['userid', 'cdcountry', 'qty_shops', 'total_orders', 'not_weekend',
       'weekend', 'lunes', 'martes', 'miercoles', 'jueves', 'viernes',
       'sabado', 'domingo', 'lunch', 'dinner', 'shop_list', 'filter_tr', 'search',
       'order_through_other', 'hamburguesas', 'pizzas', 'empanadas', 'helados',
       'sushi', 'pollo', 'milanesas', 'sandwiches', 'other']].merge(seg_may, how="left", on="userid")